# LGBM + bayesian-optimization

In [ ]:
!pip install bayesian-optimization

In [ ]:
import os
import gc
import re
import pickle
import joblib
import pandas as pd
import numpy as np

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
from bayes_opt import BayesianOptimization

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score
from lightgbm import LGBMClassifier
from collections import Counter
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import PolynomialFeatures

SEED = 42

In [ ]:
from google.colab import drive
import pandas as pd

# Google Drive를 마운트
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 파일 경로 설정
file_path = '/content/drive/My Drive/데이콘/train.csv'
file_path_2 = '/content/drive/My Drive/데이콘/test.csv'

# pandas를 사용하여 데이터 파일 불러오기
train_data = pd.read_csv(file_path)
test_data = pd.read_csv(file_path_2)

In [ ]:
train_data.describe()

,대출금액,연간소득,부채_대비_소득_비율,총계좌수,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수
count,9.629400e+04,9.629400e+04,96294.000000,96294.000000,96294.000000,9.629400e+04,9.629400e+04,96294.000000,96294.000000
mean,1.830400e+07,9.392672e+07,19.379590,25.304827,0.345681,8.225035e+05,4.282282e+05,54.380584,0.005805
std,1.032908e+07,9.956871e+07,33.569559,12.088566,0.919119,1.027745e+06,4.402111e+05,1414.769218,0.079966
min,1.200000e+06,0.000000e+00,0.000000,4.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000
25%,1.020000e+07,5.760000e+07,12.650000,17.000000,0.000000,3.075720e+05,1.346160e+05,0.000000,0.000000
50%,1.680000e+07,7.800000e+07,18.740000,24.000000,0.000000,5.976960e+05,2.870040e+05,0.000000,0.000000
75%,2.400000e+07,1.128000e+08,25.540000,32.000000,0.000000,1.055076e+06,5.702160e+05,0.000000,0.000000
max,4.200000e+07,1.080000e+10,9999.000000,169.000000,30.000000,4.195594e+07,5.653416e+06,75768.000000,4.000000


In [ ]:
# 새로운 feature 형성
train_data['원금_대비_이자_비율'] = (train_data['총상환이자'].replace(0,0.00000001)) / (train_data['총상환원금'].replace(0,0.00000001))
test_data['원금_대비_이자_비율'] = (test_data['총상환이자'].replace(0,0.00000001)) / (test_data['총상환원금'].replace(0,0.00000001))

train_data['대출_대비_원금_비율'] = (train_data['총상환원금'].replace(0,0.00000001)) / (train_data['대출금액'] )
test_data['대출_대비_원금_비율'] = (test_data['총상환원금'].replace(0,0.00000001)) / (test_data['대출금액'] )

train_data['대출_대비_이자_비율'] = (train_data['총상환이자'].replace(0,0.00000001)) / (train_data['대출금액'] )
test_data['대출_대비_이자_비율'] = (test_data['총상환이자'].replace(0,0.00000001)) / (test_data['대출금액'] )

train_data['연간소득_대비_원금_비율'] = (train_data['총상환원금'].replace(0,0.00000001)) / (train_data['연간소득'].replace(0,0.00000001))
test_data['연간소득_대비_원금_비율'] = (test_data['총상환원금'].replace(0,0.00000001)) / (test_data['연간소득'].replace(0,0.00000001))


# 대출기간을 숫자로 변환 (예: '36 months' -> 36, '60 months' -> 60)
train_data['대출기간_숫자'] = train_data['대출기간'].str.extract('(\d+)').astype(int)
test_data['대출기간_숫자'] = test_data['대출기간'].str.extract('(\d+)').astype(int)

In [ ]:
conti_col = ['대출기간_숫자' , '원금_대비_이자_비율', '대출_대비_원금_비율', '대출_대비_이자_비율', '총상환이자','총상환원금' ]
p_train_data = train_data[conti_col]
# PolynomialFeatures를 사용하여 2차 다항 특성 생성
# 변수들간의 상호작용을 전처리
poly = PolynomialFeatures(degree=2)
poly_features = poly.fit_transform(p_train_data)
polyed = pd.DataFrame(poly_features)
train_data = pd.concat([train_data,polyed],axis=1)

p_test_data = test_data[conti_col]
poly_features_test = poly.transform(p_test_data)
polyed_test = pd.DataFrame(poly_features_test)
test_data = pd.concat([test_data, polyed_test], axis=1)

In [ ]:
train_data = train_data.drop(columns=['ID','근로기간', '연체계좌수', '대출기간'])
test_ids = test_data['ID']
test_data = test_data.drop(columns=['ID','근로기간', '연체계좌수', '대출기간'])

In [ ]:
# X's & Y Split
Y = train_data['대출등급']
X = train_data.drop(columns=['대출등급'])

In [ ]:
# 선택된 수치형 변수를 사용하여 이상치 탐지를 수행
numerical_features = ['대출_대비_원금_비율', '대출_대비_이자_비율']

# LOF 모델 생성
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.01)

# 이상치 탐지 실행
lof_labels = lof.fit_predict(X[numerical_features])

# 이상치 결과 추가
X['LOF_Outlier'] = lof_labels

# 이상치를 제거하기 위해 LOF_Outlier 열이 1인 행만 필터링
X = X[X['LOF_Outlier'] == 1]

X = X.drop(['LOF_Outlier'], axis=1)

X.describe()

,대출금액,연간소득,부채_대비_소득_비율,총계좌수,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,원금_대비_이자_비율,대출_대비_원금_비율,...,18,19,20,21,22,23,24,25,26,27
count,9.533100e+04,9.533100e+04,95331.000000,95331.000000,95331.000000,9.533100e+04,9.533100e+04,95331.000000,9.533100e+04,9.533100e+04,...,9.533100e+04,9.533100e+04,9.533100e+04,9.533100e+04,9.533100e+04,95331.000000,9.533100e+04,9.533100e+04,9.533100e+04,9.533100e+04
mean,1.838934e+07,9.410941e+07,19.391955,25.321962,0.345900,8.252528e+05,4.303158e+05,51.641376,1.965850e+09,4.775235e-02,...,4.586476e-03,1.287743e-03,2.274346e+04,7.659233e+04,7.529069e-04,14872.189596,2.274346e+04,3.798658e+11,5.420227e+11,1.744648e+12
std,1.031681e+07,9.993124e+07,33.728229,12.093855,0.919222,1.031318e+06,4.412439e+05,1394.836463,4.698885e+11,4.802304e-02,...,3.195708e-02,1.609679e-03,3.563174e+04,6.469058e+05,1.042758e-03,26388.058750,3.563174e+04,8.921747e+11,1.171931e+12,1.917155e+13
min,1.200000e+06,0.000000e+00,0.000000,4.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,8.333333e-16,2.380952e-16,...,5.668934e-32,5.668934e-32,0.000000e+00,0.000000e+00,5.668934e-32,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.047000e+07,5.760000e+07,12.650000,17.000000,0.000000,3.081840e+05,1.358220e+05,0.000000,2.998103e-01,2.349294e-02,...,5.519183e-04,2.233705e-04,3.651849e+03,7.748429e+03,1.158923e-04,1640.448640,3.651849e+03,1.844762e+10,4.636048e+10,9.497738e+10
50%,1.680000e+07,7.800000e+07,18.750000,24.000000,0.000000,5.989440e+05,2.889120e+05,0.000000,4.651914e-01,4.357500e-02,...,1.898781e-03,8.772640e-04,1.264815e+04,2.423522e+04,3.799381e-04,5545.978446,1.264815e+04,8.347014e+10,1.830574e+11,3.587339e+11
75%,2.400000e+07,1.128000e+08,25.560000,32.000000,0.000000,1.056648e+06,5.727120e+05,0.000000,8.937165e-01,6.899800e-02,...,4.760724e-03,1.936246e-03,3.011446e+04,6.122613e+04,9.603077e-04,16388.027520,3.011446e+04,3.279990e+11,5.811667e+11,1.116505e+12
max,4.200000e+07,1.080000e+10,9999.000000,169.000000,30.000000,4.195594e+07,5.653416e+06,75768.000000,1.354656e+14,1.000000e+00,...,1.000000e+00,5.945570e-02,2.140405e+06,4.191192e+07,3.293004e-02,887808.679696,2.140405e+06,3.196111e+13,7.705459e+13,1.760301e+15


In [ ]:
# 선택된 수치형 변수를 사용하여 이상치 탐지를 수행
numerical_features = ['연간소득']

# LOF 모델 생성
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.01)

# 이상치 탐지 실행
lof_labels = lof.fit_predict(X[numerical_features])

# 이상치 결과 추가
X['LOF_Outlier'] = lof_labels

# 이상치를 제거하기 위해 LOF_Outlier 열이 1인 행만 필터링
X = X[X['LOF_Outlier'] == 1]

X = X.drop(['LOF_Outlier'], axis=1)

X.describe()

,대출금액,연간소득,부채_대비_소득_비율,총계좌수,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,원금_대비_이자_비율,대출_대비_원금_비율,...,18,19,20,21,22,23,24,25,26,27
count,9.437700e+04,9.437700e+04,94377.000000,94377.000000,94377.000000,9.437700e+04,9.437700e+04,94377.000000,9.437700e+04,9.437700e+04,...,9.437700e+04,9.437700e+04,9.437700e+04,9.437700e+04,9.437700e+04,94377.000000,9.437700e+04,9.437700e+04,9.437700e+04,9.437700e+04
mean,1.829814e+07,9.286653e+07,19.425403,25.265605,0.345529,8.202008e+05,4.285134e+05,52.163387,1.985721e+09,4.774912e-02,...,4.579007e-03,1.288448e-03,2.262913e+04,7.559201e+04,7.537262e-04,14826.735984,2.262913e+04,3.770744e+11,5.359996e+11,1.696339e+12
std,1.026156e+07,9.912630e+07,33.885438,12.063728,0.918896,1.011741e+06,4.398326e+05,1401.858878,4.722574e+11,4.794845e-02,...,3.180539e-02,1.608499e-03,3.518269e+04,6.226463e+05,1.044467e-03,26363.468793,3.518269e+04,8.893325e+11,1.144731e+12,1.759903e+13
min,1.200000e+06,0.000000e+00,0.000000,4.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,8.333333e-16,2.380952e-16,...,5.668934e-32,5.668934e-32,0.000000e+00,0.000000e+00,5.668934e-32,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.029000e+07,5.724000e+07,12.690000,17.000000,0.000000,3.078480e+05,1.353480e+05,0.000000,2.998150e-01,2.349273e-02,...,5.519082e-04,2.233687e-04,3.639337e+03,7.702845e+03,1.158960e-04,1634.616919,3.639337e+03,1.831908e+10,4.594541e+10,9.477039e+10
50%,1.680000e+07,7.800000e+07,18.780000,24.000000,0.000000,5.965440e+05,2.876520e+05,0.000000,4.651440e-01,4.357500e-02,...,1.898781e-03,8.772465e-04,1.256925e+04,2.422475e+04,3.799381e-04,5516.072284,1.256925e+04,8.274367e+10,1.813048e+11,3.558647e+11
75%,2.400000e+07,1.104000e+08,25.600000,32.000000,0.000000,1.051068e+06,5.701800e+05,0.000000,8.974023e-01,6.899800e-02,...,4.760724e-03,1.939134e-03,2.999256e+04,6.101467e+04,9.608177e-04,16310.108268,2.999256e+04,3.251052e+11,5.738157e+11,1.104744e+12
max,4.200000e+07,1.080000e+10,9999.000000,169.000000,30.000000,3.672116e+07,5.653416e+06,75768.000000,1.354656e+14,1.000000e+00,...,1.000000e+00,5.945570e-02,2.140405e+06,3.555000e+07,3.293004e-02,887808.679696,2.140405e+06,3.196111e+13,7.705459e+13,1.348444e+15


In [ ]:
# 선택된 수치형 변수를 사용하여 이상치 탐지를 수행
numerical_features = ['부채_대비_소득_비율']

# LOF 모델 생성
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.05)

# 이상치 탐지 실행
lof_labels = lof.fit_predict(X[numerical_features])

# 이상치 결과 추가
X['LOF_Outlier'] = lof_labels

# 이상치를 제거하기 위해 LOF_Outlier 열이 1인 행만 필터링
X = X[X['LOF_Outlier'] == 1]

X = X.drop(['LOF_Outlier'], axis=1)

X.describe()

,대출금액,연간소득,부채_대비_소득_비율,총계좌수,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,원금_대비_이자_비율,대출_대비_원금_비율,...,18,19,20,21,22,23,24,25,26,27
count,8.965800e+04,8.965800e+04,89658.000000,89658.000000,89658.000000,8.965800e+04,8.965800e+04,89658.000000,8.965800e+04,8.965800e+04,...,8.965800e+04,8.965800e+04,8.965800e+04,8.965800e+04,8.965800e+04,89658.000000,8.965800e+04,8.965800e+04,8.965800e+04,8.965800e+04
mean,1.830474e+07,9.289384e+07,19.175060,25.258616,0.346472,8.215790e+05,4.275939e+05,50.123268,2.090236e+09,4.781004e-02,...,4.594122e-03,1.286598e-03,2.260566e+04,7.572060e+04,7.499684e-04,14757.813195,2.260566e+04,3.755042e+11,5.356992e+11,1.699172e+12
std,1.026404e+07,9.985250e+07,8.488509,12.042970,0.923047,1.012023e+06,4.389417e+05,1384.519693,4.845262e+11,4.804527e-02,...,3.190799e-02,1.601414e-03,3.503119e+04,6.231064e+05,1.039151e-03,26263.494953,3.503119e+04,8.873308e+11,1.142231e+12,1.766502e+13
min,1.200000e+06,6.000000e+06,0.000000,4.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,8.333333e-16,2.380952e-16,...,5.668934e-32,5.668934e-32,0.000000e+00,0.000000e+00,5.668934e-32,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.035000e+07,5.760000e+07,12.860000,17.000000,0.000000,3.081840e+05,1.353360e+05,0.000000,2.998048e-01,2.349300e-02,...,5.519210e-04,2.233523e-04,3.639337e+03,7.747872e+03,1.155195e-04,1628.580183,3.639337e+03,1.831583e+10,4.588341e+10,9.497738e+10
50%,1.680000e+07,7.800000e+07,18.640000,24.000000,0.000000,5.986800e+05,2.870880e+05,0.000000,4.639654e-01,4.357992e-02,...,1.899209e-03,8.772267e-04,1.255534e+04,2.423522e+04,3.799158e-04,5504.904600,1.255534e+04,8.241952e+10,1.813194e+11,3.584177e+11
75%,2.400000e+07,1.104000e+08,25.030000,32.000000,0.000000,1.054812e+06,5.684040e+05,0.000000,8.927937e-01,6.900000e-02,...,4.761000e-03,1.920842e-03,2.997037e+04,6.113611e+04,9.602252e-04,16228.003549,2.997037e+04,3.230831e+11,5.735811e+11,1.112628e+12
max,4.200000e+07,1.080000e+10,39.960000,169.000000,30.000000,3.672116e+07,5.653416e+06,75768.000000,1.354656e+14,1.000000e+00,...,1.000000e+00,5.945570e-02,2.140405e+06,3.555000e+07,3.293004e-02,887808.679696,2.140405e+06,3.196111e+13,7.705459e+13,1.348444e+15


In [ ]:
# 선택된 수치형 변수를 사용하여 이상치 탐지를 수행
numerical_features = ['총계좌수']

# LOF 모델 생성
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.01)

# 이상치 탐지 실행
lof_labels = lof.fit_predict(X[numerical_features])

# 이상치 결과 추가
X['LOF_Outlier'] = lof_labels

# 이상치를 제거하기 위해 LOF_Outlier 열이 1인 행만 필터링
X = X[X['LOF_Outlier'] == 1]

X = X.drop(['LOF_Outlier'], axis=1)

X.describe()


,대출금액,연간소득,부채_대비_소득_비율,총계좌수,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,원금_대비_이자_비율,대출_대비_원금_비율,...,18,19,20,21,22,23,24,25,26,27
count,8.955700e+04,8.955700e+04,89557.000000,89557.000000,89557.000000,8.955700e+04,8.955700e+04,89557.000000,8.955700e+04,8.955700e+04,...,8.955700e+04,8.955700e+04,8.955700e+04,8.955700e+04,8.955700e+04,89557.000000,8.955700e+04,8.955700e+04,8.955700e+04,8.955700e+04
mean,1.829891e+07,9.285514e+07,19.171889,25.186663,0.346450,8.210590e+05,4.273279e+05,49.643825,2.092594e+09,4.780426e-02,...,4.586686e-03,1.286238e-03,2.258647e+04,7.545031e+04,7.496665e-04,14744.856344,2.258647e+04,3.750130e+11,5.350026e+11,1.690585e+12
std,1.026196e+07,9.988258e+07,8.488870,11.844671,0.923075,1.008196e+06,4.386410e+05,1376.001590,4.847993e+11,4.797358e-02,...,3.179977e-02,1.599466e-03,3.492927e+04,6.176011e+05,1.038797e-03,26231.671536,3.492927e+04,8.856042e+11,1.139068e+12,1.748861e+13
min,1.200000e+06,6.000000e+06,0.000000,4.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,8.333333e-16,2.380952e-16,...,5.668934e-32,5.668934e-32,0.000000e+00,0.000000e+00,5.668934e-32,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.035000e+07,5.760000e+07,12.850000,17.000000,0.000000,3.081720e+05,1.352040e+05,0.000000,2.998048e-01,2.349300e-02,...,5.519210e-04,2.233503e-04,3.639337e+03,7.740701e+03,1.155095e-04,1627.920810,3.639337e+03,1.828012e+10,4.588341e+10,9.496998e+10
50%,1.680000e+07,7.800000e+07,18.630000,24.000000,0.000000,5.981520e+05,2.868600e+05,0.000000,4.639567e-01,4.358000e-02,...,1.899216e-03,8.772225e-04,1.255306e+04,2.423522e+04,3.799158e-04,5495.821350,1.255306e+04,8.228866e+10,1.811721e+11,3.577858e+11
75%,2.400000e+07,1.104000e+08,25.030000,32.000000,0.000000,1.054368e+06,5.680080e+05,0.000000,8.927895e-01,6.900000e-02,...,4.761000e-03,1.918231e-03,2.996320e+04,6.113611e+04,9.602148e-04,16197.963471,2.996320e+04,3.226331e+11,5.727217e+11,1.111692e+12
max,4.200000e+07,1.080000e+10,39.960000,92.000000,30.000000,3.672116e+07,5.653416e+06,75768.000000,1.354656e+14,1.000000e+00,...,1.000000e+00,5.945570e-02,2.140405e+06,3.555000e+07,3.293004e-02,887808.679696,2.140405e+06,3.196111e+13,7.705459e+13,1.348444e+15


In [ ]:
# 숫자형(Integer), 범주형(Categorical) 변수 분할
numerical_list=[]
categorical_list=[]

for i in X.columns :
  if X[i].dtypes == 'O' :
    categorical_list.append(i)
  else :
    numerical_list.append(i)

print("categorical_list :", categorical_list)
print("numerical_list :", numerical_list)

categorical_list : ['주택소유상태', '대출목적']
numerical_list : ['대출금액', '연간소득', '부채_대비_소득_비율', '총계좌수', '최근_2년간_연체_횟수', '총상환원금', '총상환이자', '총연체금액', '원금_대비_이자_비율', '대출_대비_원금_비율', '대출_대비_이자_비율', '연간소득_대비_원금_비율', '대출기간_숫자', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]


In [ ]:
encoders = {}
for col in categorical_list:
    encoder = LabelEncoder()
    encoder.fit(X[col])  # 원래 트레인 데이터에 대해 fit
    encoders[col] = encoder

In [ ]:
remaining_indices = X.index
Y_filtered = Y.loc[remaining_indices]
# X의 현재 인덱스를 사용하여 train_test_split 호출
train_idx, valid_idx = train_test_split(X.index, test_size=0.3,random_state=SEED)

# 반환된 인덱스를 사용하여 X와 Y를 분할
x_train = X.loc[train_idx]
x_valid = X.loc[valid_idx]
y_train = Y_filtered.loc[train_idx]
y_valid = Y_filtered.loc[valid_idx]

In [ ]:
X_test = test_data[x_train.columns]

In [ ]:
def safe_transform(col, encoder, default_val=-1):
    labels = list(encoder.classes_)
    return col.apply(lambda x: encoder.transform([x])[0] if x in labels else default_val)

for col in categorical_list:
    # 각 컬럼에 대해 저장된 LabelEncoder를 사용하여 훈련 데이터와 검증 데이터를 변환합니다.
    x_train[col] = encoders[col].transform(x_train[col])
    x_valid[col] = encoders[col].transform(x_valid[col])

# 이제 테스트 데이터에 safe_transform을 적용합니다.
for col in categorical_list:
    X_test[col] = safe_transform(X_test[col], encoders[col])


In [ ]:
x_train = x_train.values
# 샘플링 전략 설정
sampling_strategy = "auto"
smote = SMOTE(sampling_strategy=sampling_strategy, random_state=SEED)
x_train, y_train = smote.fit_resample(x_train, y_train)

# 샘플링 후 클래스 분포 확인
print("SMOTE class distribution:", Counter(y_train))

SMOTE class distribution: Counter({'A': 18692, 'E': 18692, 'C': 18692, 'B': 18692, 'D': 18692, 'F': 18692, 'G': 18692})


In [ ]:
from bayes_opt import BayesianOptimization

# Objective Function
def lgbm_eval(max_depth, n_estimators):
    params = {
        'n_estimators': int(round(n_estimators)),
        'learning_rate': 0.05,
        'max_depth': int(round(max_depth)),
        'num_leaves': 31,
        'reg_alpha': 0.3,
        'objective': 'multiclassova',
        'random_state': SEED
    }

    clf = LGBMClassifier(**params)
    clf.fit(x_train, y_train)

    y_pred = clf.predict(x_valid)
    f1 = f1_score(y_valid, y_pred, average='macro')
    return f1

# Bayesian Optimization
optimizer = BayesianOptimization(
    f=lgbm_eval,
    pbounds={'max_depth': (7, 15), 'n_estimators': (100, 600)},
    random_state=SEED,
    verbose=2
)

optimizer.maximize(init_points=5, n_iter=20)


|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 18692, number of negative: 112152
[LightGBM] [Info] Number of positive: 18692, number of negative: 112152
[LightGBM] [Info] Number of positive: 18692, number of negative: 112152
[LightGBM] [Info] Number of positive: 18692, number of negative: 112152
[LightGBM] [Info] Number of positive: 18692, number of negative: 112152
[LightGBM] [Info] Number of positive: 18692, number of negative: 112152
[LightGBM] [Info] Number of positive: 18692, number of negative: 112152
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.021152 seconds.
You can set `force_row_wise=true

KeyboardInterrupt: 

In [ ]:
best_params = optimizer.max['params']
print(best_params)


In [ ]:
# n_estimators
n_tree = [399, 400]
# learning_rate
l_rate = [0.05]
# max_depth
m_depth = [12, 13]
# reg_alpha
L1_norm = [0.3]
# num_leaves
num_lvs = [31]

# Modeling
save_n = []
save_l = []
save_m = []
save_L1 = []
save_num_lv = []
f1_score_ = []

cnt = 0

for n in n_tree:
    for l in l_rate:
        for m in m_depth:
            for L1 in L1_norm:
                for nl in num_lvs:
                    print(">>> {} <<<".format(cnt))
                    cnt +=1
                    print("n_estimators : {}, learning_rate : {}, max_depth : {}, reg_alpha : {}, num_leaves : {}".format(n, l, m, L1, nl))
                    model = LGBMClassifier(n_estimators=n, learning_rate=l, max_depth=m, reg_alpha=L1, num_leaves=nl, n_jobs=-1, objective='multiclassova')
                    model.fit(x_train, y_train)

                    # Train Acc
                    y_pre_train = model.predict(x_train)
                    cm_train = confusion_matrix(y_train, y_pre_train)
                    print("Train Confusion Matrix")
                    print(cm_train)
                    print("Train Acc : {}".format(np.diag(cm_train).sum()/cm_train.sum()))
                    print("Train F1-Score : {}".format(f1_score(y_train, y_pre_train, average= 'macro')))

                    # Test Acc
                    y_pre_test = model.predict(x_valid)
                    cm_test = confusion_matrix(y_valid, y_pre_test)
                    print("Test Confusion Matrix")
                    print(cm_test)
                    print("Test Acc : {}".format(np.diag(cm_test).sum()/cm_test.sum()))
                    print("Test F1-Score : {}".format(f1_score(y_valid, y_pre_test, average= 'macro')))
                    print("-----------------------------------------------------------------------")
                    print("-----------------------------------------------------------------------")
                    save_n.append(n)
                    save_l.append(l)
                    save_m.append(m)
                    save_L1.append(L1)
                    save_num_lv.append(nl)
                    f1_score_.append(f1_score(y_valid, y_pre_test, average='macro'))

>>> 0 <<<
n_estimators : 599, learning_rate : 0.09, max_depth : 13, reg_alpha : 0.2, num_leaves : 31
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 18692, number of negative: 112152
[LightGBM] [Info] Number of positive: 18692, number of negative: 112152
[LightGBM] [Info] Number of positive: 18692, number of negative: 112152
[LightGBM] [Info] Number of positive: 18692, number of negative: 112152
[LightGBM] [Info] Number of positive: 18692, number of negative: 112152
[LightGBM] [Info] Number of positive: 18692, number of negative: 112152
[LightGBM] [Info] Number of positive: 18692, number of negative: 112152
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.129515 seconds.
You can set `force_col_wise=tru

In [ ]:
print(">>> {} <<<\nBest Test f1-score : {}\nBest n_estimators : {}\nBest Learning Rate : {}\nBest Max_depth : {}\nBest L1-norm : {}\n Best num_leaves : {}".format(np.argmax(f1_score_),
                                                                                                                                            f1_score_[np.argmax(f1_score_)],
                                                                                                                                            save_n[np.argmax(f1_score_)],
                                                                                                                                            save_l[np.argmax(f1_score_)],
                                                                                                                                            save_m[np.argmax(f1_score_)],
                                                                                                                                            save_L1[np.argmax(f1_score_)],
                                                                                                                                            save_num_lv[np.argmax(f1_score_)]))

>>> 0 <<<
Best Test f1-score : 0.9161603150324922
Best n_estimators : 599
Best Learning Rate : 0.09
Best Max_depth : 13
Best L1-norm : 0.2
 Best num_leaves : 31


In [ ]:
best_model = LGBMClassifier(n_estimators=save_n[np.argmax(f1_score_)], learning_rate=save_l[np.argmax(f1_score_)],
                           max_depth=save_m[np.argmax(f1_score_)], reg_alpha=save_L1[np.argmax(f1_score_)], num_leaves=save_num_lv[np.argmax(f1_score_)], objective='Multiclassova',
                           random_state=SEED)
best_model.fit(x_train, y_train)

# Train Acc
y_pre_train = best_model.predict(x_train)
cm_train = confusion_matrix(y_train, y_pre_train)
print("Train Confusion Matrix")
print(cm_train)
print("Train Acc : {}".format(np.diag(cm_train).sum()/cm_train.sum()))
print("Train F1-Score : {}".format(f1_score(y_train, y_pre_train, average='macro')))

# Test Acc
y_pre_test = best_model.predict(x_valid)
cm_test = confusion_matrix(y_valid, y_pre_test)
print("Test Confusion Matrix")
print(cm_test)
print("Test Acc : {}".format(np.diag(cm_test).sum()/cm_test.sum()))
print("Test F1-Score : {}".format(f1_score(y_valid, y_pre_test, average='macro')))

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 18692, number of negative: 112152
[LightGBM] [Info] Number of positive: 18692, number of negative: 112152
[LightGBM] [Info] Number of positive: 18692, number of negative: 112152
[LightGBM] [Info] Number of positive: 18692, number of negative: 112152
[LightGBM] [Info] Number of positive: 18692, number of negative: 112152
[LightGBM] [Info] Number of positive: 18692, number of negative: 112152
[LightGBM] [Info] Number of positive: 18692, number of negative: 112152
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.021324 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM]

In [ ]:
import joblib

# 모델을 파일로 저장
model_path = '/content/drive/My Drive/데이콘/best_model.joblib'
joblib.dump(best_model, model_path)


['/content/drive/My Drive/데이콘/best_model.joblib']

In [ ]:
predictions = best_model.predict(X_test)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [ ]:
# 예측 결과와 ID 열 결합
submission = pd.DataFrame({'ID': test_ids, '대출등급': predictions})

# 제출 파일로 저장
submission.to_csv('submission.csv', index=False)

In [ ]:
submission

,ID,대출등급
0,TEST_00000,B
1,TEST_00001,B
2,TEST_00002,A
3,TEST_00003,C
4,TEST_00004,C
...,...,...
64192,TEST_64192,D
64193,TEST_64193,D
64194,TEST_64194,D
64195,TEST_64195,C


In [ ]:
from google.colab import files
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>